In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# This notebook is a modified copy of 
# https://www.kaggle.com/nickdudaev/talkingdata-mobile-user-demographics/categorizing-labels
# unfortunately the fork and download didn't work

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re


# Original author's foreword

## Foreword:

I realize that there maybe some mistakes in generalizing labels. 
I also have left some categories unchanged because they have big counts and because I am not sure how to categorize them.
For example, *_Tencent_* label. I googled and they produce different apps, messangers, games and so on. How to categorize them
Another example, *_Industry tag_*. I left it unchanged.

Regarding minor mistakes in categorization I thought I will not care much about it at the moment because:
* in general labels list is very messy. I have strong impression that these was generated from few disparate lists which makes it difficult to work with in a more reasonable way
* Few mistakes in categorization should not have a big impact on final picture when I look at it from *_gender age_* angle

Further comments are for each category, those explain my logic

I also tried to avoid creating too 'narrow' categories because it could be that this category would not be that representative

** P.S. This can be by no doubt further improved **

In [ ]:
labels = pd.read_csv("files/label_categories.csv")
app_labels = pd.read_csv("files/app_labels.csv")
apps = pd.merge(app_labels, labels, how='left', on='label_id')

In [ ]:
apps['general_groups'] = apps['category']

In [ ]:
def to_Games(x):
    # I have gone through 'almost' manually through existing categories and came up with this non-elegant regex
    # there are a lot of games, so we can split the games category further
    # add gambling
    if re.search("(?i)((poker)|(gambl)|(roulette)|(casino))"
                 ,x) is not None:
        return('Game_Gambling')
    # add action games
    patter = re.compile("(?i)((action)|(game))")
    if re.search("(?i)((shoot)|(fight)|(tank)|(snipe)|(zombie)|(military)|(moba)|(martial)|(senki)|(ninja))"
                 ,x) is not None or len(pattern.findall(x))>1:
        return('Game_Action')
    # add rpg and strategy games
    if re.search('(RPG)|(MMO)|(ARPG)|([sS]trategy)|(Warcraft)|(tower)|(role)', x) is not None:
        return('Game_RPG')
    # add sport games, be aware of activity apps
    patter = re.compile("(?i)((sport)|(game)|(fishing)|(box)|(motorcyle)|(parkour)|(flight))") 
    if re.search("(?i)((ball)|(tennis)|(billard)|(sailing)|(racing)|(rowing)|(aircraft))"
                 ,x) is not None or len(patter.findall(x))>1:
        return('Game_Sports')
    # puzzle and skill
    pattern = re.compile("(?i)((gem)|(game)|(table)|(board)|(card)|(word)|(education)|(adventures))") 
    if re.search('(?i)((chess)|(puzzle)|(math)|(dice)|(simulat)|(tactic)|(majiang))', x) is not None or len(pattern.findall(x))>1:
        return('Game_Skill')
    # then just categorise other games
    if re.search("(?i)(game)",x) is not None:
        return('Game_Other')
    else:
        return(x)